<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [18]:
import pandas as pd
import re
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import wordnet
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import warnings
from nltk.stem import WordNetLemmatizer
warnings.filterwarnings('ignore')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\slepnev.py\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\slepnev.py\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\slepnev.py\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\slepnev.py\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\slepnev.py\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [71]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv('toxic_comments.csv')

data.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


Выводы:
1. Текст на английсокм языке
2. Текст необходимо леммитизировать и очистить

In [72]:
lemmatizer = WordNetLemmatizer()
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(text):
    s = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return(' '.join(s))


def clear_text(text):
    pattern = r'[^a-zA-Z]'
    text = re.sub(pattern, " ", text)
    
    return " ".join(text.split())


In [ ]:
%%time
data['lemma'] = data['text'].apply(lambda x: lemmatize(clear_text(x)))

CPU times: user 21min 23s, sys: 1min 21s, total: 22min 45s
Wall time: 22min 54s


In [3]:
#Посмотрим количество Nan (если в ячейке были толкьо символы)
print(data['lemma'].isna().sum())
#уберем
data = data[data['lemma'].notna()]

11


In [11]:
#Скорее всего, придется рассматривать несколько моделей, разобъем на train, valid, test
_,test = train_test_split(data, test_size=0.2, random_state=42)
train, valid = train_test_split(_, test_size=0.1, random_state=42)
X_train, y_train = train['lemma'], train['toxic']
X_valid, y_valid = valid['lemma'], valid['toxic']
X_test, y_test = test['lemma'], test['toxic']

## Обучение

In [26]:
#Попробуем использовать 2 модели.
stopwords = set(nltk_stopwords.words('english'))

pipe_lr = Pipeline([('vectorizer', TfidfVectorizer(stop_words=stopwords)),
                    ('LR', LogisticRegression(random_state=42))])

pipe_knn = Pipeline([('vectorizer', TfidfVectorizer(stop_words=stopwords)),
                    ('knn', KNeighborsClassifier())])

In [27]:
lr_param_grid = [{'LR__max_iter': [75, 100, 120],
                  'LR__C' :[5, 10, 20]}]

knn_param_grid = [{'knn__n_neighbors': [3, 5]}]


In [30]:
cv = KFold(3, shuffle=True, random_state=42)
lr_grid_search = GridSearchCV(estimator=pipe_lr,
                 param_grid=lr_param_grid,
                 scoring='f1',
                 cv=cv)

knn_grid_search = GridSearchCV(estimator=pipe_knn,
                         param_grid=knn_param_grid,
                         scoring='f1',
                         cv=cv)

In [61]:
%%time
lr_grid_search.fit(X_train, y_train)

CPU times: total: 9min 37s
Wall time: 3min 44s


GridSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                                 

In [64]:
lr_grid_search.best_params_

{'LR__C': 20, 'LR__max_iter': 120}

In [63]:
#Посмотрим результат на валидационной выборке
pred = lr_grid_search.best_estimator_.predict(X_valid)
score = f1_score(y_valid, pred)
print(score)

0.7621527777777778


In [43]:
%%time
knn_grid_search.fit(X_train, y_train)

CPU times: total: 10min 3s
Wall time: 10min 4s


GridSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                                 

In [57]:
knn_grid_search.best_params_

{'knn__n_neighbors': 3}

In [60]:
#Посмотрим результат на валидационной выборке
pred = knn_grid_search.best_estimator_.predict(X_valid)
score = f1_score(y_valid, pred)
print(score)

0.3217665615141956


Вывод: Логистическая регрессия показала лучшие результаты на валидационной выборке, ее и будем проверять на тестовой выборке

In [65]:
#Проверим на тестовой выборке
pred = lr_grid_search.best_estimator_.predict(X_test)
score = f1_score(y_test, pred)
print(score)

0.7762867334805503


## Выводы

1. Для леммитизации текста очень хорошо себя показала модель pymorphy2.MorphAnalyzer(). Файл обработался за 3 минуты.
2. Для обработки такого файла на локальном компьютере у меня подошла только модель логистической регрессии и метод ближайших соседей. Никакими другими моделями достичь результата не получилось. Необходимы расширенные вычислительные мощности
3. Удалось достичь требуемого результата - метрика F1 свыше 0,75.Улучшить модель удалось через введение коэффициента регуляризации. С увеличением "штрафа" за сложность модели вырасает ее точность. Но разница между С=10 и С=20 незначительная.  

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [X]  Весь код выполняется без ошибок
- [X]  Ячейки с кодом расположены в порядке исполнения
- [X]  Данные загружены и подготовлены
- [X]  Модели обучены
- [X]  Значение метрики *F1* не меньше 0.75
- [X]  Выводы написаны